# Glasses Detector Demo (Coming Soon...)

## Setup

This simply installs the minimum required **Python** version  (`3.12`) and the latest **PyTorch** version (_Nightly_ for compatibility). If you want, you can change the environment type before running the notebook to support GPU acceleration: `Runtime` $\to$ `Change runtime type`.

> **Note**: it may take several minutes to execute the cell because installing **PyTorch** generally takes a while.

In [ ]:
!sudo apt-get update -qq -y && sudo apt-get install python3.12 &> /dev/null
!sudo update-alternatives --quiet --install /usr/bin/python3 python3 /usr/bin/python3.12 1
!curl -sS https://bootstrap.pypa.io/get-pip.py | python3.12 &> /dev/null
!pip install --pre torch torchvision ipython --index-url https://download.pytorch.org/whl/nightly/cu121 &> /dev/null
!git clone -q https://github.com/mantasu/glasses-detector && pip install ./glasses-detector &> /dev/null

In [ ]:
from IPython.display import Image

## Command Line

### Classification

In [ ]:
!glasses-detection -t classification:anyglasses

### Detection

### Segmentation

## Python Scripts (Coming Soon...)